Keong Jo Hsi 1001685  
Homework 4 (Coding Part 1)

In [310]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models
import torchvision.transforms as transforms

import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook

import PIL.Image as Image


In [383]:
def parsefile(f):
    return np.genfromtxt(f, dtype=None, encoding='utf-8')

In [387]:
class dataset_flowers(Dataset):

    def __init__(self, img_dir='./jpg/', transform=None, mode='train'):
        self.img_dir = img_dir
        self.transform = transform

        if mode == 'train':
            dataset = parsefile('trainfile_10.txt')
        elif mode == 'test':
            dataset = parsefile('valfile.txt')
        self.dataset = dataset

    def __getitem__(self, index):

        jpg, label = self.dataset[index]
        imgfile = os.path.join(self.img_dir, jpg)
        img = Image.open(imgfile).convert('RGB')

        if self.transform:
            augmented_img = self.transform(img)
        else:
            trans = transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
            augmented_img = trans(img)

        return {'img': augmented_img, 'label': label}

    def __len__(self):
        return len(self.dataset)

#### With Pretraining

In [388]:
pretrained_model = models.resnet18(pretrained=True)
optimizer = optim.SGD(params=model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

In [389]:
dataset = dataset_flowers(mode='train')
dl = DataLoader(dataset=dataset, batch_size=5)

correct = 0
denominator = 0
running_loss = 0

pretrained_model.train()

for data_dict in dl: 
    optimizer.zero_grad()
    
    data = data_dict['img']  # [5,3,244,244], FloatTensor
    label = data_dict['label'].long() # [5], LongTensor
    
    output = pretrained_model(data) # [5,1000], FloatTensor
#     print('output', output)
    
    pred = (torch.argmax(output, dim=1)) # [5], FloatTensor
#     print('pred', pred)
#     print('label', label)

    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    
    # minibatch_loss * minibatch_size
    running_loss += loss.item() * label.shape[0]
    
    correct += sum(torch.eq(pred,label)).item()
    denominator += label.shape[0]
        
accuracy = float(correct) / denominator
print('accuracy = {}, loss = {}'.format(accuracy, running_loss/denominator))


accuracy = 0.0, loss = 9.26341199874878


#### Without Pretraining

In [388]:
model = models.resnet18(pretrained=False)
optimizer = optim.SGD(params=model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

In [389]:
dataset = dataset_flowers(mode='train')
dl = DataLoader(dataset=dataset, batch_size=5)

correct = 0
denominator = 0
running_loss = 0

model.train()

for data_dict in dl: 
    optimizer.zero_grad()
    
    data = data_dict['img']  # [5,3,244,244], FloatTensor
    label = data_dict['label'].long() # [5], LongTensor
    
    output = pretrained_model(data) # [5,1000], FloatTensor
#     print('output', output)
    
    pred = (torch.argmax(output, dim=1)) # [5], FloatTensor
#     print('pred', pred)
#     print('label', label)

    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    
    # minibatch_loss * minibatch_size
    running_loss += loss.item() * label.shape[0]
    
    correct += sum(torch.eq(pred,label)).item()
    denominator += label.shape[0]
        
accuracy = float(correct) / denominator
print('accuracy = {}, loss = {}'.format(accuracy, running_loss/denominator))


accuracy = 0.0, loss = 9.26341199874878


#### Pretraining with last layer removed

In [392]:
pretrained_model = models.resnet18(pretrained=True)
optimizer = optim.SGD(params=model.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()


<bound method Module.named_modules of Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (

In [389]:
dataset = dataset_flowers(mode='train')
dl = DataLoader(dataset=dataset, batch_size=5)

correct = 0
denominator = 0
running_loss = 0

pretrained_model.train()

for data_dict in dl: 
    optimizer.zero_grad()
    
    data = data_dict['img']  # [5,3,244,244], FloatTensor
    label = data_dict['label'].long() # [5], LongTensor
    
    output = pretrained_model(data) # [5,1000], FloatTensor
#     print('output', output)
    
    pred = (torch.argmax(output, dim=1)) # [5], FloatTensor
#     print('pred', pred)
#     print('label', label)

    loss = criterion(output, label)
    loss.backward()
    optimizer.step()
    
    # minibatch_loss * minibatch_size
    running_loss += loss.item() * label.shape[0]
    
    correct += sum(torch.eq(pred,label)).item()
    denominator += label.shape[0]
        
accuracy = float(correct) / denominator
print('accuracy = {}, loss = {}'.format(accuracy, running_loss/denominator))


accuracy = 0.0, loss = 9.26341199874878
